In [5]:
###########import packages##########
import tensorflow as tf
import keras
from keras import optimizers
from keras import regularizers
from keras import backend as K
from keras.callbacks import EarlyStopping
from keras.callbacks import TensorBoard
from keras.constraints import max_norm
from keras.models import Sequential 
from keras.layers import Dense 
from keras.layers import Dropout 
from keras.models import Model
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier 
from keras.wrappers.scikit_learn import KerasRegressor
from keras.constraints import maxnorm 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xgboost as xgb
import lightgbm
import catboost
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from xgboost import plot_importance
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn import ensemble
from sklearn.tree import ExtraTreeRegressor
from sklearn import svm
from sklearn import neighbors
from sklearn import tree
from sklearn.impute import SimpleImputer
from fancyimpute import KNN
from keras.callbacks import EarlyStopping 
from sklearn.metrics import r2_score
early_stopping=keras.callbacks.EarlyStopping(
 monitor="val_loss", 
 patience=20, 
 verbose=0, 
 mode="auto"
)
%matplotlib
###########wrapping root mean square error for later calls##########
def compute_mae_mse_rmse(target,prediction):
    error = []
    for i in range(len(target)):
        error.append(target[i] - prediction[i])
    squaredError = []
    absError = []
    for val in error:
        squaredError.append(val * val)  # target-prediction之差平方
        absError.append(abs(val))  # 误差绝对值
    mae=sum(absError)/len(absError)  # 平均绝对误差MAE
    mse=sum(squaredError)/len(squaredError)  # 均方误差MSE
    RMSE=sum(absError)/len(absError)
    R2=r2_score(target,prediction)
    return mae,mse,RMSE,R2
###########loading data##########
fdata=pd.read_csv('database_filled.csv',encoding="gbk")
raw_data=fdata.loc[:,[                      
                      'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)',#13
                      'Half Wave Potential in Acid Condition (vs. RHE)'#the classification target#14
                        ]]

###########data standardization##########
standardized_data = (raw_data-np.mean(raw_data,axis=0))/np.std(raw_data,axis=0)

###########defining a wrapper function for later call from each machine learning algorithms##########
raw_input=standardized_data.iloc[:,0:14]
raw_output=standardized_data.iloc[:,14]
X=raw_input.values.astype(np.float32)
y=raw_output.values.astype(np.float32)
###########fix random seed for reproducability##########
seed=71
###########train test splitting##########
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15,random_state=seed)
raw_input_global=raw_data.iloc[:,0:14]
raw_output_global=raw_data.iloc[:,14]
###########wrap up fuction for later call for OPTIMIZATION##########
def evaluate(pre_2,real_2):
    pre_2=np.array(pre_2)
    real_2=np.array(real_2)
    pre_2_series=pd.Series(pre_2)
    real_2_series=pd.Series(real_2)
    return rmse(pre_2,real_2), round(pre_2_series.corr(real_2_series), 3)
def compare(list_name,limit):
    judge=1
    for a in list_name:
        if a < limit:
            judge=judge*1
        else:
            judge=judge*0
    return judge
def generate_arrays_from_file(path):
    while True:
        with open(path) as f:
            for line in f:
                # create numpy arrays of input data
                # and labels, from each line in the file
                x1, x2, y = process_line(line)
                yield ({'input_1': x1, 'input_2': x2}, {'output': y})
for neurons1 in [50,100,200]:
    for epochs_number in [200,400,800]:
        for learning_rate_search in [0.0001,0.0005,0.001,0.005,0.01]:
            for dropout_rate in [0,0.25,0.5]:
                for batch_size_number in [16,32]:
                    for reg in [0,0.0001,0.001]:
                        regularizer=keras.regularizers.l2(reg)
                        activation1='relu'
                        ###########keras ANN model construction##########
                        model = Sequential() 
                        model.add(Dense(neurons1, input_dim=14, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))
                        model.add(Dense(neurons1, input_dim=neurons1, kernel_initializer='random_normal',
                                        bias_initializer='random_normal',activation=activation1,kernel_regularizer=regularizer)) 
                        model.add(Dropout(dropout_rate))
                        model.add(Dense(1, input_dim=neurons1, activation='linear'))
                        adam=optimizers.Adam(lr=learning_rate_search)
                        model.compile(loss='mse', optimizer=adam)
                        print('training...')
                        model.fit(X_train, y_train,verbose=0, batch_size=batch_size_number,epochs=epochs_number,validation_split=0.2,callbacks=[early_stopping])
                        result=model.predict(X_test)
                        ###########get RMSE and R2##########
                        x_prediction_07=result*np.std(raw_output_global,axis=0)+np.mean(raw_output_global,axis=0)
                        y_real_07=np.std(raw_output_global,axis=0)*y_test+np.mean(raw_output_global,axis=0)
                        x_prediction_07_series=pd.Series(x_prediction_07[:,0])
                        y_real_07_series=pd.Series(y_real_07)
                        ###########evaluating the regression quality##########
                        corr_ann = round(x_prediction_07_series.corr(y_real_07_series), 5)
                        error_val= compute_mae_mse_rmse(x_prediction_07[:,0],y_real_07)
                        print('scatter corr',corr_ann,'scatter error',error_val)
                        print(neurons1,epochs_number,learning_rate_search,dropout_rate,batch_size_number,reg)
                        x_y_x=np.arange(0.5,1,0.1)
                        x_y_y=np.arange(0.5,1,0.1)
                        fig = plt.figure()
                        ax = fig.add_subplot(111)
                        ax.scatter(x_prediction_07[:,0],y_real_07,color='red',label='Artificial Neural Network')
                        ax.plot(x_y_x,x_y_y)
                        plt.legend()
                        plt.xlabel(u"Predicted_Half_Wave_Potential V (vs. RHE)")
                        plt.ylabel(u"Real_Half_Wave_Potential V (vs. RHE)")
                        plt.savefig('%s %s %s %s %s %s.png' %(neurons1,epochs_number,learning_rate_search,dropout_rate,batch_size_number,reg))
                        K.clear_session()    

Using matplotlib backend: Qt5Agg
training...
scatter corr 0.89601 scatter error (0.026076770880643058, 0.0009583367868598918, 0.026076770880643058, 0.6226861543000515)
50 200 0.0001 0 32 0


In [ ]:
import eli5
def base_model():
    tmodel = Sequential() 
    tmodel.add(Dense(50, input_dim=14, kernel_initializer='random_normal',
                    bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0))) 
    tmodel.add(Dropout(0))
    tmodel.add(Dense(50, input_dim=neurons1, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0))) 
    tmodel.add(Dropout(0))
    tmodel.add(Dense(1, input_dim=50, activation='linear'))
    adam=optimizers.Adam(lr=0.0001)
    tmodel.compile(loss='mse', optimizer=adam) 
    return tmodel
from eli5.sklearn import PermutationImportance
my_model = KerasRegressor(build_fn=base_model,nb_epoch=200, batch_size=32, verbose= False)    
my_model.fit(X_train, y_train)
perm = PermutationImportance(my_model,random_state=1,n_iter=10).fit(X_train,y_train)
eli5.show_weights(perm)

In [ ]:
smodel = Sequential() 
smodel.add(Dense(50, input_dim=14, kernel_initializer='random_normal',
                bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0))) 
smodel.add(Dropout(0))
smodel.add(Dense(50, input_dim=neurons1, kernel_initializer='random_normal',
            bias_initializer='random_normal',activation='relu',kernel_regularizer=keras.regularizers.l2(0))) 
smodel.add(Dropout(0))
smodel.add(Dense(1, input_dim=50, activation='linear'))
adam=optimizers.Adam(lr=0.0001)
smodel.compile(loss='mse', optimizer=adam) 
print('training...')
smodel.fit(X_train, y_train,verbose=0, batch_size=32,epochs=200)

In [ ]:
import shap
# %matplotlib
standardized_data_SHAP = standardized_data
SHAP_INPUT=standardized_data_SHAP.iloc[:,0:14]
SHAP_OUTPUT=standardized_data_SHAP.iloc[:,14]
X_SHAP=SHAP_INPUT.values.astype(np.float32)
y_SHAP=SHAP_OUTPUT.values.astype(np.float32)
explainer = shap.DeepExplainer(smodel,X_SHAP)
cols=[ 
              'Zn Content (wt. %)',#0
                      'Co Cotent (wt. %)',#1
                      'Fe Cotent (wt. %)',#2
                      'O Content (at. %)',#4
                      'S Content (at. %)',#5
                      'Content of Pyridinic Species (at. %)',#5
                      'Content of Graphitic Species (at. %)',#6
                      'Content of Oxidized Species (at. %)',#7
                      'Raman ID/IG Ratio',#8
                      'BET Surface Area (m2 g-1)',#9
                      'Micropore Ratio',#10
                      'Mesopore Ratio',#11
                      'Catalyst Loading on RDE/RRDE(mg cm-2)',#12
                      'Rotating Speed (rpm)'
                        ]
shap_values = explainer.shap_values(X_SHAP)
print(type(shap_values))

In [ ]:
%matplotlib
shap.summary_plot(shap_values[0], SHAP_INPUT)